In [1]:
import numpy as np
from scipy.signal import convolve2d 
from os import path, getcwd, system, getenv
from astropy.io import fits
from time import perf_counter
import urllib.request
import os

import matplotlib
import math
import matplotlib.pyplot as plt
from matplotlib import cm
import glob
import matplotlib.gridspec as gridspec

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.backends.backend_pdf import PdfPages

import sys,inspect
currentdir = path.dirname(path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = path.dirname(currentdir)
sys.path.append(parentdir)
# from deconv import cube_deconv
# from ppxf_wrap import ppxf_wrap

# import warnings

# if True:
#     print('Set warnings.filterwarnings as "ignore"')
#     print('Warning Message will not be printed')
#     warnings.filterwarnings("ignore")

from astropy.convolution import convolve, Gaussian1DKernel, Box1DKernel
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from scipy import constants
import pandas as pd

# import plotly.express as px
from astroquery.mast import Observations


from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u
from astropy.nddata.utils import Cutout2D

from astropy.wcs import WCS
import astropy

from urllib.error import HTTPError
from astroquery.skyview import SkyView
import pickle


computer = 1
maindir = "C:/Users/maxri/Desktop/Classes 4-1/Research/fuse"
obn = 90
skydowni ="./images_redOb{}".format(obn); 

In [56]:
#Create Object List/
#Save to Object Tables

catalogdir = "C:/Users/maxri/Desktop/Classes 4-1/Research/data/VI_129_fuse.dat.fits"
catalog = fits.open(catalogdir)

catheader = catalog[0]; catlist = catalog[1];

dflist = pd.DataFrame(catlist.data)



catalog.close()


obs=dflist.query("Class == {} and Avail=='y'".format(obn));

dflist



obsn1 = []
for i in obs.iterrows():
    b = [i[1][1].split()[0],i[1][2], i[1][3]] 
    obsn1.append(b)
    

    
obsn1

s1 = maindir+"/tbls/Objects{}.tbl".format(obn); 

#Reading Index
indexdir = f"{maindir}/tbls/{obn}index.txt"
indexdir

ipd = pd.read_table(indexdir, header = None)

obsn1




[['HS-1140+2711', 175.726, 26.9162],
 ['HS-1310+4308', 198.203, 42.8768],
 ['A1795', 207.219, 26.5931],
 ['HS-0834+1509', 129.304, 14.9883],
 ['VIRGOI', 187.705, 12.3919],
 ['VIRGOII', 187.78, 12.3961],
 ['VIRGOIII', 187.806, 12.3694],
 ['COMAI', 194.954, 27.9628],
 ['A2597', 351.332, -12.1242],
 ['HE-1347-2457', 207.662, -25.2047],
 ['HE-2217-2818', 335.028, -28.0565],
 ['HS-2337+1845', 354.936, 19.0309],
 ['HS-0119+1432', 20.4835, 14.8066],
 ['HS-1549+1919', 237.969, 19.1843],
 ['HS-0035+4405', 9.4679, 44.359],
 ['HE-2156-4020', 329.978, -40.0973],
 ['HS-1345+2855', 207.018, 28.6737],
 ['HE-1338-0724', 205.264, -7.6635],
 ['HS-1104+0452', 166.785, 4.6049],
 ['HS-1110+1537', 168.219, 15.3567],
 ['HS-1104+4259', 166.77, 42.7278],
 ['HS-1140+2711', 175.726, 26.9162],
 ['HS-1140+2711', 175.726, 26.9162],
 ['HS-1700+6416', 255.252, 64.2025],
 ['HS-1700+6416', 255.252, 64.2025],
 ['ABELL3112', 49.4904, -44.2382],
 ['ABELL3112', 49.4904, -44.2382],
 ['HS-1700+6416', 255.252, 64.2025],
 ['HS

In [70]:
catalogdir2 = "C:/Users/maxri/Desktop/Classes 4-1/Research/data/VII_1B_catalog.dat.fits"
catalog2 = fits.open(catalogdir2)

catheader2 = catalog2[0]; catlist2 = catalog2[1];

dflist2 = pd.DataFrame(catlist2.data)

ra2 = dflist2.RAh/24 + dflist2.RAm/60; 

dec2 = dflist2.DEd + dflist2.DEm/60



SyntaxError: f-string: unmatched '[' (701954338.py, line 12)

In [ ]:
minsep = 0.1
pairs = []
for i in range(len(obsn1)):
    if ipd[0][i] == 0:
        continue
    c1 = SkyCoord(obsn1[i][1],obsn1[i][2], unit=u.deg)
    for j in range(len(ra2)):
        c2 = SkyCoord(ra2[i],dec2[i], unit = u.deg)
        if c1.separation(c2).deg < minsep:
            a1 = dflist2["NGC"][j]; a2 = dflist2["m_NGC"][j]; a3 = dflist2["Type"][j]
            pairs.append([f"Fuse{obsn1[i][0]}",f"NGC{a1} m{a2} Type{a3}"])
            
pairs

131.8504488425875